Import yearly cleanup script

In [1]:
import yearly_cleanup as yc

Input your cleanup year

In [ ]:
year = yc.get_year()

Retrieve all the subdirectories matching your cleanup year

In [ ]:
yc.get_yearly_releases(year)

- Create the new yearly folder
- Move the subdirectories matching your cleanup year to the new yearly folder

In [ ]:
yearly_path = yc.create_year_folder(year)
yc.move_yearly_releases(yearly_path, yc.release_folders)